<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/LLMadv-kata4/GemiAikata4ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 newspaper3k google-generativeai

import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import newspaper
import json
from datetime import datetime

In [ ]:
!pip install lxml[html_clean]

In [ ]:
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=api_key)
# Initialize the model
model = genai.GenerativeModel("gemini-pro")


In [ ]:
# Function to summarize the article
def summarize_article(text):
    prompt = f"Summarize this news article:\n\n{text}"
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# Extract content from the article's URL
def extract_news(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch article: {response.status_code}")

    soup = BeautifulSoup(response.text, "html.parser")

    # Try multiple strategies to extract content
    paragraphs = soup.find_all("p")  # News articles are commonly inside <p> tags
    article_text = "\n".join([para.get_text() for para in paragraphs])

    return article_text if article_text else "No content extracted."


In [ ]:
# Classify the article based on its content
def classify_article(text):
    prompt = f"Identify the category of this news article (e.g., politics, finance, sports, technology):\n\n{text}"
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# Create a summary JSON file for the article
def create_summary_json(url):
    content = extract_news(url)
    summary = summarize_article(content)
    category = classify_article(content)

    summary_data = {
        "date": datetime.today().strftime("%d %B %Y"),
        "summary": summary,
        "articleType": category.strip()
    }

    # Save to JSON file
    with open("news_summary.json", "w") as file:
        json.dump(summary_data, file, indent=4)

    return summary_data

In [ ]:
# Ask a question based on the news summary
def ask_question(question, json_file="news_summary.json"):
    with open(json_file, "r") as file:
        news_data = json.load(file)

    prompt = f"Based on the following news summary, answer this question:\n\nSummary: {news_data['summary']}\n\nQuestion: {question}"
    response = model.generate_content(prompt)

    return response.text

In [ ]:
# Example usage
news_url = "https://www.thehindu.com/elections/delhi-assembly/middle-class-voters-gig-workers-auto-drivers-slum-dwellers-are-all-fed-up-with-aap-says-baijayant-panda/article69155949.ece"
content = extract_news(news_url)
print(content[:500])  # Print first 500 characters

January 30, 2025e-Paper

									The Hindu On Books
									Books of the week, reviews, excerpts, new titles and features.


									Data Point
									Decoding the headlines with facts, figures, and numbers


									First Day First Show
									News and reviews from the world of cinema and streaming.


									Health Matters
									Ramya Kannan writes to you on getting to good health, and staying there


									The View From India
									Looking at World Affairs from the Indian pe


In [ ]:
summary = create_summary_json(news_url)
print("News Summary:", summary)

News Summary: {'date': '03 February 2025', 'summary': 'BJP\'s Baijayant Panda asserts that Delhi residents will not continue the trend of voting for different parties in Lok Sabha and Assembly elections. He dismisses concerns about not declaring a chief ministerial candidate, highlighting their success in bridging similar gaps in Odisha. Panda emphasizes the party\'s welfare promises backed by Prime Minister Narendra Modi\'s guarantee, including income support for women and initiatives in education and infrastructure. He refutes criticisms of mirroring AAP\'s "revri" schemes, accusing AAP of corruption and failed promises. Panda believes the Congress has weakened and targets middle-class voters and gig workers who are disillusioned with AAP\'s governance.', 'articleType': 'politics (general)'}


In [ ]:
# Asking a question based on the summary of the article
question = "What is the main topic of the article?"
answer = ask_question(question)
print("Answer:", answer)

Answer: The main topic of the article is BJP's Lok Sabha election campaign strategy in Delhi.
